In [1]:
import json
import os
from pyquaternion import Quaternion
import pandas as pd
import pickle
import numpy as np

In [2]:
EXPERIMENT = False

In [3]:
if EXPERIMENT:
    suffix = '_experiment'
else:
    suffix = ''

In [4]:
full_data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/'

In [5]:
base_dir = full_data_dir
folder = 'v1.0-trainval'

In [6]:
# try:
#     os.remove(os.path.join(base_dir, f'annotation{suffix}.csv'))
# except FileNotFoundError:
#     pass

# try:
#     os.remove(os.path.join(base_dir, f'annotation{suffix}.pickle'))
# except FileNotFoundError:
#     pass

# try:
#     os.remove(os.path.join(base_dir, f'sample_data{suffix}.csv'))
# except FileNotFoundError:
#     pass

# try:
#     os.remove(os.path.join(base_dir, f'sample_data{suffix}.pickle'))
# except FileNotFoundError:
#     pass

In [7]:
with open(os.path.join(base_dir, folder, 'calibrated_sensor.json')) as f:
    calibrated_sensor_json = json.load(f)

with open(os.path.join(base_dir, folder, 'category.json')) as f:
    category_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample.json')) as f:
    sample_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_data.json')) as f:
    sample_data_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_annotation.json')) as f:
    sample_annotation_json = json.load(f)

with open(os.path.join(base_dir, folder, 'instance.json')) as f:
    instance_json = json.load(f)

with open(os.path.join(base_dir, folder, 'scene.json')) as f:
    scene_json = json.load(f)

with open(os.path.join(base_dir, folder, 'ego_pose.json')) as f:
    ego_pose_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sensor.json')) as f:
    sensor_json = json.load(f)

In [8]:
files = os.listdir(os.path.join(full_data_dir, 'experiment_data'))

In [9]:
import math
from typing import List

def normalizeAngle(angle) -> float:
    while angle > math.pi:
        angle -= math.tau
    while angle < -math.pi:
        angle += math.tau
    assert -math.pi <= angle <= math.pi
    return angle

def get_heading(rotation):
    yaw = rotation.yaw_pitch_roll[0]
    return normalizeAngle(yaw)

def get_heading_from_north(rotation):
    yaw = rotation.yaw_pitch_roll[0]
    return normalizeAngle(yaw - (math.pi / 2))

rot = Quaternion(axis=[1, 0, 0], angle=np.pi / 2)
def get_camera_heading(rotation):
    return -get_heading(rot.rotate(rotation)) + math.pi / 2
    
def get_camera_position(
    camera_translation: List[float],
    ego_translation: List[float],
    ego_rotation: List[float]
) -> np.ndarray:
    rotated_offset = Quaternion(ego_rotation) \
        .rotate(np.array(camera_translation))
    return np.array(ego_translation) + rotated_offset

In [10]:
files_set = set(files)

In [11]:
if EXPERIMENT:
    sample_data_filter = [s for s in sample_data_json if s['filename'].split('/')[2] in files_set]
else:
    sample_data_filter = sample_data_json
# sample_data_filter = [s for s in sample_data_json if 'CAM_FRONT/' in s['filename']]

In [12]:
sample_data_filter[0]

{'token': 'bddd80ae33ec4e32b27fdb3c1160a30e',
 'sample_token': 'e93e98b63d3b40209056d129dc53ceee',
 'ego_pose_token': 'bddd80ae33ec4e32b27fdb3c1160a30e',
 'calibrated_sensor_token': '7781065816974801afc4dcdaf6acf92c',
 'timestamp': 1531883530440378,
 'fileformat': 'pcd',
 'is_key_frame': True,
 'height': 0,
 'width': 0,
 'filename': 'samples/RADAR_FRONT/n015-2018-07-18-11-07-57+0800__RADAR_FRONT__1531883530440378.pcd',
 'prev': '',
 'next': '90df03ad4710427aabb5f88fe049df2e'}

In [13]:
sample_tokens = set([s['sample_token'] for s in sample_data_filter])

In [14]:
ego_pose_tokens = set([s['ego_pose_token'] for s in sample_data_filter])

In [15]:
calibrated_sensor_tokens = set([s['calibrated_sensor_token'] for s in sample_data_filter])

In [16]:
sample_filter = [
    {
        'sample_token': s['token'],
        'scene_token': s['scene_token']
    }
    for s in sample_json
    if s['token'] in sample_tokens
]
len(sample_filter)

34149

In [17]:
scene_tokens = set([s['scene_token'] for s in sample_filter])

In [18]:
calibrated_sensor_filter = [
    {
        'calibrated_sensor_token': c['token'],
        'camera_translation': c['translation'],
        'camera_rotation': c['rotation'],
        'camera_intrinsic': c['camera_intrinsic']
    }
    for c in calibrated_sensor_json
    if c['token'] in calibrated_sensor_tokens
]
len(calibrated_sensor_filter)

10200

In [19]:
ego_pose_filter = [
    {
        'ego_pose_token': e['token'],
        'ego_translation': e['translation'],
        'ego_rotation': e['rotation']
    }
    for e in ego_pose_json
    if e['token'] in ego_pose_tokens
]
len(ego_pose_filter)

2631083

In [20]:
scene_filter = [
    {
        'scene_token': s['token'],
        'scene_name': s['name'],
    }
    for s in scene_json
    if s['token'] in scene_tokens
]
len(scene_filter)

850

In [21]:
sample_map = {
    s['sample_token']: s
    for s in sample_filter
}
calibrated_sensor_map = {
    c['calibrated_sensor_token']: c
    for c in calibrated_sensor_filter
}
ego_pose_map = {
    e['ego_pose_token']: e
    for e in ego_pose_filter
}
scene_map = {
    s['scene_token']: s
    for s in scene_filter
}

def s_map(s):
    sample = sample_map[s['sample_token']]
    calibrated_sensor = calibrated_sensor_map[s['calibrated_sensor_token']]
    ego_pose = ego_pose_map[s['ego_pose_token']]
    ego_heading = get_heading_from_north(Quaternion(ego_pose['ego_rotation']))
    camera_heading = get_camera_heading(Quaternion(calibrated_sensor['camera_rotation']))
    ret = {
        **s,
        **sample,
        **calibrated_sensor,
        **ego_pose,
        **scene_map[sample['scene_token']],
        'ego_heading': ego_heading * 180 / math.pi,
        'camera_heading': normalizeAngle(camera_heading + ego_heading) * 180 / math.pi,
        'camera_translation_absolute': get_camera_position(
            calibrated_sensor['camera_translation'],
            ego_pose['ego_translation'],
            ego_pose['ego_rotation']
        )
    }
    del ret['ego_pose_token']
    del ret['calibrated_sensor_token']
    del ret['fileformat']
    del ret['height']
    del ret['width']
    del ret['prev']
    del ret['next']
    del ret['scene_token']
    return ret

sample_data_res = [*map(s_map, sample_data_filter)]

len(sample_data_res)

2631083

In [22]:
sample_annotation_filter = [
    sa
    for sa in sample_annotation_json
    if sa['sample_token'] in sample_tokens
]
len(sample_annotation_filter)

1166187

In [23]:
instance_tokens = set([
    sa['instance_token']
    for sa in sample_annotation_filter
])

instance_filter = [
    {
        'instance_token': i['token'],
        'category_token': i['category_token']
    }
    for i in instance_json
    if i['token'] in instance_tokens
]
len(instance_filter)

64386

In [24]:
category_tokens = set([
    i['category_token']
    for i in instance_filter
])

category_filter = [
    {
        'category_token': c['token'],
        'category': c['name']
    }
    for c in category_json
    if c['token'] in category_tokens
]
len(category_filter)

23

In [25]:
instance_map = {
    i['instance_token']: i
    for i in instance_filter
}
category_map = {
    c['category_token']: c
    for c in category_filter
}

def sa_map(sa):
    instance = instance_map[sa['instance_token']]
    ret = {
        **sa,
        **instance,
        **category_map[instance['category_token']],
        'heading': (get_heading_from_north(Quaternion(sa['rotation']))) * 180 / math.pi
    }
    
    del ret['visibility_token']
    del ret['attribute_tokens']
    del ret['prev']
    del ret['next']
    del ret['num_lidar_pts']
    del ret['num_radar_pts']
    del ret['category_token']
    
    return ret

sample_annotation_res = [*map(sa_map, sample_annotation_filter)]
len(sample_annotation_res)

1166187

In [26]:
sample_data_final = pd.DataFrame(sample_data_res)

In [27]:
sample_annotation_final = pd.DataFrame(sample_annotation_res)

In [28]:
df_sample_data_keyframe = sample_data_final[sample_data_final["is_key_frame"]][
    ["token", "sample_token"]
]
sample_annotation_final = sample_annotation_final.set_index("sample_token").join(
    df_sample_data_keyframe.set_index("sample_token"), on="sample_token", rsuffix="_sample_data"
)

In [29]:
sample_data_final = sample_data_final.sort_values(by=['scene_name', 'timestamp'])

In [30]:
sample_data_final["frame_order"] = 0
scene = ""
i = 1
for index, row in sample_data_final.iterrows():
    if row['scene_name'] != scene:
        scene = row['scene_name']
        sample_data_final.loc[index, "frame_order"] = 1
        i = 2
    else:
        sample_data_final.loc[index, "frame_order"] = i
        i += 1


In [31]:
print(len(sample_data_final))

2631083


In [32]:
print(len(sample_annotation_final))

13994244


In [33]:
with open(os.path.join(base_dir, f"sample_data{suffix}.pickle"), "wb") as f:
    pickle.dump(sample_data_final, f)
with open(os.path.join(base_dir, f"annotation{suffix}.pickle"), "wb") as f:
    pickle.dump(sample_annotation_final, f)

In [34]:
sample_data_final.to_csv(os.path.join(base_dir, f"sample_data{suffix}.csv"), index=False)

In [35]:
sample_annotation_final.to_csv(os.path.join(base_dir, f"annotation{suffix}.csv"), index=False)

In [36]:
calibrated_sensor = [
    {
        **c,
        **[s for s in sensor_json if s['token'] == c['sensor_token']][0]
    }
    for c in calibrated_sensor_json
]
calibrated_sensor = [
    c
    for c in calibrated_sensor
    if c['modality'] == 'camera'
]
rot = Quaternion(axis=[1, 0, 0], angle=np.pi / 2)
[
    normalizeAngle(get_heading(rot.rotate(Quaternion(c['rotation'])))) * 180 / math.pi
    for c in calibrated_sensor
    if c['channel'] == "CAM_FRONT"
]

[89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,


In [37]:
sample_data_final

,token,sample_token,timestamp,is_key_frame,filename,camera_translation,camera_rotation,camera_intrinsic,ego_translation,ego_rotation,scene_name,ego_heading,camera_heading,camera_translation_absolute,frame_order
2850,24332e9c554a406f880430f17771b608,e93e98b63d3b40209056d129dc53ceee,1531883530404844,True,samples/CAM_FRONT_LEFT/n015-2018-07-18-11-07-5...,"[1.52387798135, 0.494631336551, 1.50932822144]","[0.6757265034669446, -0.6736266522251881, 0.21...","[[1272.5979470598488, 0.0, 826.6154927353808],...","[1010.1055895106443, 610.6258204233582, 0.0]","[-0.7537076134272889, -0.007693345098279576, 0...",scene-0001,-7.832026,47.328759,"[1009.8186635148887, 612.1676965406358, 1.5444...",1
1657,020d7b4f858147558106c504f7f31bef,e93e98b63d3b40209056d129dc53ceee,1531883530412470,True,samples/CAM_FRONT/n015-2018-07-18-11-07-57+080...,"[1.70079118954, 0.0159456324149, 1.51095763913]","[0.4998015430569128, -0.5030316162024876, 0.49...","[[1266.417203046554, 0.0, 816.2670197447984], ...","[1010.1102882349232, 610.6567106479714, 0.0]","[-0.7530285141171715, -0.007718682910458633, 0...",scene-0001,-7.713726,-7.388005,"[1010.3184772563701, 612.3090687734056, 1.5499...",2
1936,16d39ff22a8545b0a4ee3236a0fe1c20,e93e98b63d3b40209056d129dc53ceee,1531883530420339,True,samples/CAM_FRONT_RIGHT/n015-2018-07-18-11-07-...,"[1.5508477543, -0.493404796419, 1.49574800619]","[0.2060347966337182, -0.2026940577919598, 0.68...","[[1260.8474446004698, 0.0, 807.968244525554], ...","[1010.1150499070123, 610.6886117142394, 0.0]","[-0.752322806136323, -0.007744708072671614, 0....",scene-0001,-7.590939,-63.991801,"[1010.8050264478901, 612.1259954776945, 1.5309...",3
2103,ec7096278e484c9ebe6894a2ad5682e9,e93e98b63d3b40209056d129dc53ceee,1531883530427893,True,samples/CAM_BACK_RIGHT/n015-2018-07-18-11-07-5...,"[1.0148780988, -0.480568219723, 1.56239545128]","[0.12280980120078765, -0.132400842670559, -0.7...","[[1259.5137405846733, 0.0, 807.2529053838625],...","[1010.1196021097104, 610.7194357591594, 0.0]","[-0.7516415465789779, -0.007600820970155064, 0...",scene-0001,-7.472257,-118.261334,"[1010.7235435454575, 611.6273507367362, 1.5852...",4
2402,aab35aeccbda42de82b2ff5c278a0d48,e93e98b63d3b40209056d129dc53ceee,1531883530437525,True,samples/CAM_BACK/n015-2018-07-18-11-07-57+0800...,"[0.0283260309358, 0.00345136761476, 1.57910346...","[0.5037872666382278, -0.49740249788611096, -0....","[[809.2209905677063, 0.0, 829.2196003259838], ...","[1010.1256390280025, 610.760527131398, 0.0]","[-0.7507230134456502, -0.007585124591152042, 0...",scene-0001,-7.312768,172.548458,"[1010.1214297170264, 610.7532852880651, 1.5793...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2630679,12f905ed55cb431aa4330f5b508764f0,5abb17189643497f8600a187fb20aa55,1542801733437525,True,samples/CAM_BACK/n015-2018-11-21-19-58-31+0800...,"[0.0283260309358, 0.00345136761476, 1.57910346...","[0.5037872666382278, -0.49740249788611096, -0....","[[809.2209905677063, 0.0, 829.2196003259838], ...","[1529.0367956454097, 1298.6783721527465, 0.0]","[0.6792136990312807, 0.0049900395429761804, 0....",scene-1110,175.566609,-4.572164,"[1529.0663063612035, 1298.5961612703038, 1.576...",3102
2628487,491cab8354c54570a561eca2a93c4197,5abb17189643497f8600a187fb20aa55,1542801733446609,True,samples/RADAR_FRONT_LEFT/n015-2018-11-21-19-58...,"[2.422, 0.8, 0.78]","[0.7171539204983457, 0.0, 0.0, 0.6969148113750...",[],"[1529.030823018331, 1298.6118732766831, 0.0]","[0.679209024472939, 0.004846355276846213, 0.01...",scene-1110,175.565699,-94.434301,"[1529.6551947257194, 1296.1106766069533, 0.684...",3103
2630913,bcbdddd82a9d4228a60205a8a9c4aded,5abb17189643497f8600a187fb20aa55,1542801733447423,True,samples/CAM_BACK_LEFT/n015-2018-11-21-19-58-31...,"[1.03569100218, 0.484795032713, 1.59097014818]","[0.6924185592174665, -0.7031619420114925, -0.1...","[[1256.7414812095406, 0.0, 792.1125740759628],...","[1529.030287400523, 1298.6059147402107, 0.0]","[0.6792086049065644, 0.004833479999277509, 0.0...",scene-1110,175.565